# DeepAR Notes and Questions

Do we need to hot encode city? Wouldn't latitude and longitude be better data in terms of percentages of fire within a given location?

If cities were given a binary value, wouldn't the predictions be more suscetible to prediciting fires ONLY in the cities that had fires previously, and give a 0 percent prediction on cities that weren't in our training data?

But I guess if the user is giving us a location, hot encoding the locations would be better.

BUT.... web-services could take the location, web-scrape its latitude and longitude and have them pass those values to us to make a prediction.

Just thinking out loud, maybe something we could consider.

# Setup
Importing Sagemaker and setting file parameters. DO NOT RUN THESE UNTIL WE ARE READY TO SEND THIS NOTEBOOK INTO AWS SAGEMAKER!

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/Wildfire-DeepAR"  # change to your desired S3 prefix
region = sess.boto_region_name

role = sagemaker.get_execution_role()

Import Necessary Files and packages

In [ ]:
import boto3
import io
import json
import time

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import pandas as pd

import s3fs

# Setting Up Data
Download the cleaned CSV file here

Refer to tutorial for assistance

Mak will implmenet the preparation of the data by adding non"true" fire results to the cleaned csv.

To accomplish this, we need to loop through every fire or weather station
for each weather station, we loop throughout the range of dates... yes every single one, and add a 1 to 
the target attribute if there was a fire on that specific day, 0 if otherwise

This will allow the DeepAR learning algorithm to gain a probability of a fire in a particular city.

NOTE, this might prove challenging if the input for location is converted to latitude and longitude but im certain 
it's doable

In [1]:
# Download the cleaned CSV file here
# Refer to tutorial for assistance
# Mak will implmenet the preparation of the data by adding non"true" fire results to the cleaned csv
# To accomplish this, we need to loop through every fire or weather station
# For each weather station, we loop throughout the range of dates... yes every single one, and add a 1 to 
# the target attribute if there was a fire on that specific day, 0 if otherwise
# This will allow the DeepAR learning algorithm to gain a probability of a fire in a particular city.
# NOTE, this might prove challenging if the input for location is converted to latitude and longitude but im certain 
# it's doable

In [ ]:
# download the data
s3 = boto3.client("s3")
datafile = "fireWithWeather.csv"

df = pd.read_csv(datafile)

# Hopefully it's clean

In [ ]:
s3_output_path = "{}/{}/output".format(bucket, prefix)
train_data_path = "s3://{}/{}/train/train.json".format(bucket, prefix)
test_data_path = "s3://{}/{}/test/test.json".format(bucket, prefix)

def series_to_obj(ts, cat=None):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    if cat:
        obj["cat"] = cat
    return obj


def series_to_jsonline(ts, cat=None):
    return json.dumps(series_to_obj(ts, cat))

with open("train.json", "w") as fp:
    for ts in violation_list_training:
        fp.write(series_to_jsonline(ts))
        fp.write("\n")
! aws s3 mv train.json $train_data_path

with open("test.json", "w") as fp:
    for ts in violation_list:
        fp.write(series_to_jsonline(ts))
        fp.write("\n")
! aws s3 mv test.json $test_data_path

# Training Algorithm 

Copied from the AWS GitHub training repo, modified for our needs 

In [ ]:
# Hyperparameters

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker import image_uris


container = image_uris.retrieve(region=region, framework="forecasting-deepar")

deepar = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    use_spot_instances=True,  # use spot instances
    max_run=1800,  # max training time in seconds
    max_wait=1800,  # seconds to wait for spot instance
    output_path="s3://{}/{}".format(bucket, s3_output_path),
    sagemaker_session=sess,
)
freq = "D"
context_length = 30

deepar.set_hyperparameters(
    time_freq=freq, context_length=str(context_length), prediction_length=str(prediction_length)
)

hyperparameter_ranges = {
    "mini_batch_size": IntegerParameter(100, 400),
    "epochs": IntegerParameter(200, 400),
    "num_cells": IntegerParameter(30, 100),
    "likelihood": CategoricalParameter(["negative-binomial", "student-T"]),
    "learning_rate": ContinuousParameter(0.0001, 0.1),
}

objective_metric_name = "test:RMSE"

tuner = HyperparameterTuner(
    deepar,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=10,
    strategy="Bayesian",
    objective_type="Minimize",
    max_parallel_jobs=10,
    early_stopping_type="Auto",
)

s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data="s3://{}/{}/train/".format(bucket, prefix), content_type="json"
)
s3_input_test = sagemaker.inputs.TrainingInput(
    s3_data="s3://{}/{}/test/".format(bucket, prefix), content_type="json"
)

tuner.fit({"train": s3_input_train}, include_cls_metadata=False)
tuner.wait()